In [20]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions

In [68]:
using MacroTools

In [115]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    
    x_prev = x_prior
    
    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P) where { 
            dependencies = WithInboundFunctionalDependencies((2, ), (NormalMeanVariance(0.0, 1.0), )) 
        }
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [131]:
P = 1.0
n = 2

data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [135]:
function inference(data, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, 1.0, P, options = (limit_stack_depth = 500, ));

    x_buffer  = buffer(Marginal, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, x_buffer)
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return getvalues(x_buffer)
end

inference (generic function with 1 method)

In [138]:
@rule NormalMeanVariance(:μ, Marginalisation) (m_out::PointMass, m_μ::NormalMeanVariance, m_v::PointMass, ) = begin 
    return NormalMeanVariance(mean(m_out), mean(m_v))
end

In [139]:
inference(data, x0_prior, P)

2-element Vector{Marginal}:
 Marginal(NormalMeanVariance{Float64}(μ=0.9164184913484523, v=0.4999750012499375))
 Marginal(NormalMeanVariance{Float64}(μ=1.916418491348452, v=0.4999750012499375))

In [112]:
x0_prior = NormalMeanVariance(0.0, 10000.0)
@benchmark res = inference($data, $x0_prior, $P)

BenchmarkTools.Trial: 
  memory estimate:  13.17 MiB
  allocs estimate:  215598
  --------------
  minimum time:     18.686 ms (0.00% GC)
  median time:      19.897 ms (0.00% GC)
  mean time:        22.769 ms (13.47% GC)
  maximum time:     34.298 ms (41.69% GC)
  --------------
  samples:          220
  evals/sample:     1

In [113]:
?findfirst

search: findfirst UndefinedNodeFunctionalForm



```
findfirst(A)
```

Return the index or key of the first `true` value in `A`. Return `nothing` if no such value is found. To search for other kinds of values, pass a predicate as the first argument.

Indices or keys are of the same type as those returned by [`keys(A)`](@ref) and [`pairs(A)`](@ref).

# Examples

```jldoctest
julia> A = [false, false, true, false]
4-element Vector{Bool}:
 0
 0
 1
 0

julia> findfirst(A)
3

julia> findfirst(falses(3)) # returns nothing, but not printed in the REPL

julia> A = [false false; true false]
2×2 Matrix{Bool}:
 0  0
 1  0

julia> findfirst(A)
CartesianIndex(2, 1)
```

---

```
findfirst(predicate::Function, A)
```

Return the index or key of the first element of `A` for which `predicate` returns `true`. Return `nothing` if there is no such element.

Indices or keys are of the same type as those returned by [`keys(A)`](@ref) and [`pairs(A)`](@ref).

# Examples

```jldoctest
julia> A = [1, 4, 2, 2]
4-element Vector{Int64}:
 1
 4
 2
 2

julia> findfirst(iseven, A)
2

julia> findfirst(x -> x>10, A) # returns nothing, but not printed in the REPL

julia> findfirst(isequal(4), A)
2

julia> A = [1 4; 2 2]
2×2 Matrix{Int64}:
 1  4
 2  2

julia> findfirst(iseven, A)
CartesianIndex(2, 1)
```

---

```
findfirst(pattern::AbstractString, string::AbstractString)
findfirst(pattern::AbstractPattern, string::String)
```

Find the first occurrence of `pattern` in `string`. Equivalent to [`findnext(pattern, string, firstindex(s))`](@ref).

# Examples

```jldoctest
julia> findfirst("z", "Hello to the world") # returns nothing, but not printed in the REPL

julia> findfirst("Julia", "JuliaLang")
1:5
```

---

```
findfirst(ch::AbstractChar, string::AbstractString)
```

Find the first occurrence of character `ch` in `string`.

!!! compat "Julia 1.3"
    This method requires at least Julia 1.3.


# Examples

```jldoctest
julia> findfirst('a', "happy")
2

julia> findfirst('z', "happy") === nothing
true
```

---

```
findfirst(pattern::AbstractVector{<:Union{Int8,UInt8}},
          A::AbstractVector{<:Union{Int8,UInt8}})
```

Find the first occurrence of sequence `pattern` in vector `A`.

!!! compat "Julia 1.6"
    This method requires at least Julia 1.6.


# Examples

```jldoctest
julia> findfirst([0x52, 0x62], [0x40, 0x52, 0x62, 0x63])
2:3
```


In [6]:
x0_prior = NormalMeanVariance(0.0, 10000.0)
@benchmark res = inference($data, $x0_prior, $P)

BenchmarkTools.Trial: 
  memory estimate:  12.28 MiB
  allocs estimate:  206592
  --------------
  minimum time:     14.879 ms (0.00% GC)
  median time:      16.607 ms (0.00% GC)
  mean time:        18.578 ms (11.87% GC)
  maximum time:     29.016 ms (39.59% GC)
  --------------
  samples:          269
  evals/sample:     1

In [5]:
x0_prior = NormalMeanVariance(0.0, 10000.0)
@benchmark res = inference($data, $x0_prior, $P)

BenchmarkTools.Trial: 
  memory estimate:  12.28 MiB
  allocs estimate:  206592
  --------------
  minimum time:     14.691 ms (0.00% GC)
  median time:      16.059 ms (0.00% GC)
  mean time:        18.036 ms (12.46% GC)
  maximum time:     26.821 ms (38.14% GC)
  --------------
  samples:          277
  evals/sample:     1

In [14]:
inference(data, x0_prior, P)

500-element Vector{Marginal}:
 Marginal(NormalMeanVariance{Float64}(μ=0.9887735574697237, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=1.988773557469724, v=0.0019999996000000823))
 Marginal(NormalMeanVariance{Float64}(μ=2.988773557469723, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=3.988773557469723, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=4.988773557469723, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=5.988773557469724, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=6.988773557469723, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=7.988773557469725, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=8.988773557469722, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=9.988773557469722, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=10.988773557469722, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}